In [13]:
import timm
import torch
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef

In [14]:
file_path = "/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePUUM/1. Completed_Data/CanonBeetles.csv"

canon_df = pd.read_csv(file_path)

canon_df["ImageFilePath"] = canon_df["cropped_image_path"].apply(lambda x: f"/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePUUM/CANON/{x}")

cols = ['ImageFilePath', 'ScientificName']

df = canon_df[cols]


In [15]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = timm.create_model(
    model_name="hf-hub:1aurent/resnet50.lunit_swav",
    pretrained=True,
).to(device).eval()

data_config = timm.data.resolve_model_data_config(model)

transforms = timm.data.create_transform(**data_config, is_training=False)


In [16]:
def extract_features(image_path) :
    
    image = Image.open(image_path).convert("RGB")
    
    image_tensor = transforms(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        features = model(image_tensor)
    
    return features.cpu().numpy()

In [ ]:
X = np.vstack([extract_features(img) for img in tqdm(df["ImageFilePath"])])

le = LabelEncoder()

y = le.fit_transform(df["ScientificName"])

df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df_indices, test_size=0.2, random_state=42)

test_df = df.loc[test_idx].copy()

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


In [ ]:
seed = 99

models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "SVMLinear": SVC(kernel="linear"),
    "SVMPolynomial": SVC(kernel="poly", degree=4),
    "SVMRadialBasis": SVC(kernel="rbf", degree=4),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=5),
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=seed),    
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(256, 128), activation='logistic', max_iter=200, random_state=seed)
}

predictions = {}

metrics = {}

for name, model in models.items():
    
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds
    
    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)
    
    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}
    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T

In [19]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,ScientificName,Pred_NaiveBayes,Pred_LogisticRegression,Pred_SVMLinear,Pred_SVMPolynomial,Pred_SVMRadialBasis,Pred_NearestNeighbor,Pred_RandomForest,Pred_MLP_Baseline
135,/fs/ess/PAS2136/Hawaii-2025/beetles_intake/Bee...,Mecyclothorax discedens,Mecyclothorax discedens,Mecyclothorax discedens,Mecyclothorax discedens,Mecyclothorax konanus,Mecyclothorax konanus,Mecyclothorax discedens,Mecyclothorax konanus,Mecyclothorax discedens
844,/fs/ess/PAS2136/Hawaii-2025/beetles_intake/Bee...,Mecyclothorax konanus,Mecyclothorax brunneonubiger,Mecyclothorax konanus,Mecyclothorax konanus,Mecyclothorax konanus,Mecyclothorax konanus,Mecyclothorax konanus,Mecyclothorax konanus,Mecyclothorax konanus


In [20]:
metrics_df

,Model,Accuracy,Precision,Recall,F1-Score,Balanced Acc,MCC
NaiveBayes,NaiveBayes,0.341538,0.516835,0.341538,0.379395,0.13694,0.174996
LogisticRegression,LogisticRegression,0.910769,0.897614,0.910769,0.898515,0.530086,0.862975
SVMLinear,SVMLinear,0.907692,0.900228,0.907692,0.894217,0.542556,0.857952
SVMPolynomial,SVMPolynomial,0.523077,0.628635,0.523077,0.398243,0.160902,0.223327
SVMRadialBasis,SVMRadialBasis,0.849231,0.785624,0.849231,0.808359,0.320288,0.763854
NearestNeighbor,NearestNeighbor,0.781538,0.769579,0.781538,0.754169,0.325134,0.655951
RandomForest,RandomForest,0.790769,0.740005,0.790769,0.722249,0.23727,0.669495
MLP_Baseline,MLP_Baseline,0.923077,0.916211,0.923077,0.913974,0.546411,0.882448


In [21]:
test_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/Beetle-PUUM/37.SwAV-linear-probing-species.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/Beetle-PUUM/37.SwAV-linear-probing-species-metrics.csv", index=False)